In [ ]:
import json
import h5py
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import os

from tqdm import tqdm
from coco import COCO_CLASSES

plt.rcParams["figure.figsize"] = (10, 5)

In [ ]:
ROOT_VG = "/Users/ozzy/Projects/Datasets/VisualGenome/"
with open(ROOT_VG + 'objects.json') as in1:
    objects_file = json.load(in1)

with open(ROOT_VG + 'image_data.json') as in2:
    image_data = json.load(in2)
    
with open('SYNSET_TO_NAME_MAPPER.json') as in3:
    SYNSET_TO_NAMES_MAPPER = json.load(in3)

with open('COCO_TO_VG_SYNSET_MAPPER.json') as in4:
    COCO_TO_VG_SYNSET_MAPPER = json.load(in4)
    
with open('COCO_CLASSES_IN_VG_COUNT.json') as in5:
    COCO_CLASSES_IN_VG_COUNT = json.load(in5)

all_ids = []
all_files = os.listdir(ROOT_VG + "VG_100K")
for i in all_files:
    all_ids.append(i.split(".")[0])
    
new_image_data = {}
for all_data in image_data:
    new_image_data[all_data['image_id']] = all_data

In [ ]:
COCO_TO_VG_SYNSET_MAPPER_INVERSE = {}
for p in COCO_TO_VG_SYNSET_MAPPER:
    for m in COCO_TO_VG_SYNSET_MAPPER[p]:
        if m not in COCO_TO_VG_SYNSET_MAPPER_INVERSE:
            COCO_TO_VG_SYNSET_MAPPER_INVERSE[m] = [p]
        else:
            COCO_TO_VG_SYNSET_MAPPER_INVERSE[m].append(p)

In [ ]:
# In all cases, the returned relative vector is like BOX1 <SPATIAL RELATION> BOX2
def relative_location(x1, y1, w1, h1, x2, y2, w2, h2, visualize=False):
    bins = {"left": 0, "center": 1, "right": 2, "top": 3, "bottom":4}
    rel_vector = np.zeros(len(bins))
    
    # Center
    if (abs(x1 - x2) <= w1/2) and (abs(y1 - y2)<= h1/2):
        rel_vector[bins['center']] = 1
    
    if rel_vector[bins['center']] != 1:
        # Left
        if x1 < x2:
            rel_vector[bins['left']] = 1

        # Right
        if x1 > x2:
            rel_vector[bins['right']] = 1       

        # Top
        if y1 > (y2 + h2/2):
            rel_vector[bins['top']] = 1    

        # Bottom
        if y1 < (y2 - h2/2):
            rel_vector[bins['bottom']] = 1  

    if visualize:
        for c, i in enumerate(rel_vector):
            if i == 1:
                print("RED BOX is " + list(bins.keys())[c] + " (of) BLUE BOX")

        fig, ax = plt.subplots()
        plt.scatter(x1, y1, color='r')
        rect = patches.Rectangle((x1-(w1/2), y1-(h1/2)), w1, h1, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.scatter(x2, y2, color='b')
        rect = patches.Rectangle((x2-(w2/2), y2-(h2/2)), w2, h2, linewidth=1, edgecolor='b', facecolor='none')
        ax.add_patch(rect)
        plt.show()
    
    return rel_vector

In [ ]:
# Example
relative_location(395.0, 402.5, 72, 81, 0, 30, 142, 255, visualize=True)

In [ ]:
spatial_rel_dict = {}
for c in COCO_TO_VG_SYNSET_MAPPER:
    in_dict = {} 
    for cc in COCO_TO_VG_SYNSET_MAPPER:
        # {L, C, R, T, B} stand for {“left", “center", “right", “top", “bottom"}
        # https://arxiv.org/pdf/2206.00481.pdf
        in_dict[cc] = [0, 0, 0, 0, 0, 0]
    spatial_rel_dict[c] = in_dict


In [ ]:
relevant_classes = [j for sub in list(COCO_TO_VG_SYNSET_MAPPER.values()) for j in sub]
for c, objects in tqdm(enumerate(objects_file), total=len(objects_file)):
    objects = objects['objects']        
    for object_i in objects:
        if object_i['synsets'] == []:
            continue
        object_i_id = object_i['object_id']
        valid_names_i = []
        stripped_names_i = [synset_i.split(".")[0].replace('_', ' ') for synset_i in object_i["synsets"]]

        new_stripped_names_i = []
        for name in stripped_names_i:
            for k in COCO_TO_VG_SYNSET_MAPPER:
                key_list = COCO_TO_VG_SYNSET_MAPPER[k]
                if name in key_list:
                    if name not in new_stripped_names_i:
                        new_stripped_names_i.append(name)
        if new_stripped_names_i == []:
            continue
            
        for name_i in stripped_names_i:
            if name_i in relevant_classes:
                valid_names_i.append(name_i)
        if valid_names_i == []:
            continue
        my_x_center = (object_i['w'] / 2) + object_i['x']
        my_y_center = (object_i['h'] / 2) + object_i['y']

        # Iterate over neighboring objects
        for object_j in objects:
            if object_j['synsets'] == []:
                continue
            object_j_id = object_j['object_id']

            # Skip statistics with the identical object
            if object_i_id == object_j_id:
                continue
                
            stripped_names_j = [synset_j.split(".")[0].replace('_', ' ') for synset_j in object_j["synsets"]]
            new_stripped_names_j = []
            for name in stripped_names_j:
                for k in COCO_TO_VG_SYNSET_MAPPER:
                    key_list = COCO_TO_VG_SYNSET_MAPPER[k]
                    if name in key_list:
                        if name not in new_stripped_names_j:
                            new_stripped_names_j.append(name)
            if new_stripped_names_j == []:
                continue
                
            valid_names_j = []
            for name_j in stripped_names_j:
                if name_j in relevant_classes:
                    valid_names_j.append(name_j)
            if valid_names_j == []:
                continue
                
            their_x_center = (object_j['w'] / 2) + object_j['x']
            their_y_center = (object_j['h'] / 2) + object_j['y']
            
            
            rel_vec = relative_location(my_x_center, my_y_center, object_i['w'], object_i['h'],
                                        their_x_center, their_y_center, object_j['w'], object_j['h'])

            for ss_i in new_stripped_names_i:
                inverse_list_ss_i = COCO_TO_VG_SYNSET_MAPPER_INVERSE[ss_i]
                for all_inverse_ssi in inverse_list_ss_i:
                    for ss_j in new_stripped_names_j:
                        inverse_list_ss_j = COCO_TO_VG_SYNSET_MAPPER_INVERSE[ss_j]
                        for all_inverse_ssj in inverse_list_ss_j:
                            
                            # Spatial relations
                            spatial_rel_dict[all_inverse_ssi][all_inverse_ssj][:-1] = [x + y for x, y in zip(spatial_rel_dict[all_inverse_ssi][all_inverse_ssj][:], rel_vec)]
                            
                            # How often they occur together
                            spatial_rel_dict[all_inverse_ssi][all_inverse_ssj][-1] += 1    


In [ ]:
new_one = {}
for synset in spatial_rel_dict:
    new_one[synset] = {}
    for synset_inner in spatial_rel_dict[synset]:
        new_one[synset][synset_inner] = {}
        items = spatial_rel_dict[synset][synset_inner]
        if isinstance(spatial_rel_dict[synset][synset_inner], float):
            continue
        if items[-1] == 0:
            continue
        else:
            new_one[synset][synset_inner] = [i/items[-1] for i in items[:-1]]

# For consistency
for key_i in new_one:
    for key_j in new_one[key_i]:
        if new_one[key_i][key_j] == {}:
            new_one[key_i][key_j] = [0,0,0,0,0]

In [ ]:
# Normalize
values = []
for c, row in enumerate(new_one):
    values_ = []
    for column in new_one[row]:
        values_.append(new_one[row][column])
    values.append(values_)
values = np.array(values)
values /= np.amax(values)

values_dict = {}
for c1, row in enumerate(list(COCO_CLASSES.values())[1:]):
    values_dict[row] = {}
    for c2, column in enumerate(list(COCO_CLASSES.values())[1:]):
        values_dict[row][column] = values[c1][c2].tolist()

In [ ]:
with open("relative-orientation.json", "w") as out1:
    json.dump(values_dict, out1)